Making Python faster
====

The Julia website has a set of benchmarks to show how fast it is. These are for toy functions, but serve as a nice set of examples for the basics of using `cython` and `numba` in Python.

These examples are adapted from [here](https://www.ibm.com/developerworks/community/blogs/jfp/entry/python_meets_julia_micro_performance?lang=en)

In [2]:
benchmarks = pd.read_pickle('julia_benchmarks.pic')
benchmarks.ix[:, :6]

,,Fortran,Julia,Python,R,Matlab
1,fib,0.70,2.11,77.76,533.52,26.89
2,parse_int,5.05,1.45,17.02,45.73,802.52
3,quicksort,1.31,1.15,32.89,264.54,4.92
4,mandel,0.81,0.79,15.32,53.16,7.58
5,pi_sum,1.00,1.00,21.99,9.56,1.00
6,rand_mat_stat,1.45,1.66,17.93,14.56,14.52
7,rand_mat_mul,3.48,1.02,1.14,1.57,1.12


In [145]:
import time
import random
import numpy as np

In [119]:
from numba import njit, jit

In [120]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


### Utility function

In [121]:
def timer(f, *args, **kwargs):
    start = time.clock()
    ans = f(*args, **kwargs)
    return ans, time.clock() - start

In [130]:
def report(fs, *args, **kwargs):
    ans, t = timer(fs[0], *args, **kwargs)
    print('%s: %.1f' % (fs[0].__name__, 1.0))  
    for f in fs[1:]:
        ans_, t_ = timer(f, *args, **kwargs)
        print('%s: %.1f' % (f.__name__, t/t_))

Fib
----

In [63]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

In [64]:
fib(20)

6765

In [113]:
%timeit fib(20)

100 loops, best of 3: 5.84 ms per loop


In [68]:
from functools import lru_cache

@lru_cache()
def fib_lru(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

In [114]:
%timeit fib_lru(20)

The slowest run took 92.76 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 213 ns per loop


In [115]:
%%cython -a

cpdef long fib_cython(int n):
    if n<2:
        return n
    return fib_cython(n-1) + fib_cython(n-2)

In [116]:
%timeit fib_cython(20)

10000 loops, best of 3: 129 µs per loop


In [82]:
@njit
def fib_numba(n):
    a, b = 0, 1
    for i in range(n-1):
        a, b = b, a + b
    return b

In [83]:
%timeit -r3 -n3 fib_numba(20)

The slowest run took 49263.70 times longer than the fastest. This could mean that an intermediate result is being cached 
3 loops, best of 3: 501 ns per loop


In [84]:
%load_ext cython

In [123]:
%%cython -a

def fib_cython_seq(int n):
    cdef int i
    cdef long a, b, tmp
    a = 0
    b = 1
    for i in range(n-1):
        tmp = a
        a = b
        b += tmp
    return b

In [124]:
%timeit fib_cython_seq(20)

The slowest run took 15.42 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 122 ns per loop


In [131]:
report([fib, fib_lru, fib_cython, fib_numba, fib_cython_seq], 20)

fib: 1.0
fib_lru: 1151.4
fib_cython: 26.0
fib_numba: 639.7
fib_cython_seq: 1919.0


In [442]:
fs = fib, fib_lru, fib_cython, fib_numba, fib_cython_seq
for f in fs:
    print(f(20))

6765
6765
6765
6765
6765


Parse_int
----

In [158]:
def parse_int():
    for i in range(1,1000):
        n = random.randint(0,2**31-1)
        s = hex(n)
#         if s[-1]=='L':
#             s = s[0:-1]    
        m = int(s,16)
        assert m == n

In [142]:
%timeit parse_int()

100 loops, best of 3: 4.56 ms per loop


In [159]:
def parse_int_numpy():
    for i in range(1,1000):
        n = np.random.randint(0,2**31-1)
        s = hex(n) 
        m = int(s,16)
        assert m == n

In [160]:
%timeit parse_int_numpy()

1000 loops, best of 3: 1.51 ms per loop


In [169]:
%%cython -a

import cython
import numpy as np
cimport numpy as np

def parse_int_cython():
    cdef int i, n, m
    for i in range(1,1000):
        n = np.random.randint(0,2**31-1)
        m = int(hex(n),16)
        assert m == n

In [170]:
%timeit parse_int_cython()

1000 loops, best of 3: 1.13 ms per loop


In [175]:
report([parse_int, parse_int_numpy, parse_int_cython])

parse_int: 1.0
parse_int_numpy: 2.7
parse_int_cython: 3.2


Quicksort
----

In [171]:
def qsort_kernel(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel(a, lo, j)
        lo = i
        j = hi
    return a

In [190]:
def benchmark_qsort():
    lst = [ random.random() for i in range(1,5000) ]
    qsort_kernel(lst, 0, len(lst)-1)

In [191]:
%timeit benchmark_qsort()

10 loops, best of 3: 23.8 ms per loop


In [214]:
%%cython -a

import cython
import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double[:] qsort_kernel_cython(double[:] a, int lo, int hi):
    cdef int i, j
    cdef double pivot

    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_cython(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_cython():
    lst = np.random.random(5000)
    qsort_kernel_cython(lst, 0, len(lst)-1)

In [215]:
%timeit benchmark_qsort_cython()

The slowest run took 4.02 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 735 µs per loop


In [216]:
report([benchmark_qsort, benchmark_qsort_cython])

benchmark_qsort: 1.0
benchmark_qsort_cython: 25.6


Mandel
----

In [217]:
def mandel(z):
    maxiter = 80
    c = z
    for n in range(maxiter):
        if abs(z) > 2:
            return n
        z = z*z + c
    return maxiter

def mandelperf():
    r1 = np.linspace(-2.0, 0.5, 26)
    r2 = np.linspace(-1.0, 1.0, 21)
    return [mandel(complex(r, i)) for r in r1 for i in r2]

In [219]:
%timeit mandelperf()

100 loops, best of 3: 6.9 ms per loop


In [259]:
@jit
def mandel_numba(z):
    maxiter = 80
    c = z
    for n in range(maxiter):
        if abs(z) > 2:
            return n
        z = z*z + c
    return maxiter

@jit
def mandelperf_numba():
    r1 = np.linspace(-2.0, 0.5, 26)
    r2 = np.linspace(-1.0, 1.0, 21)
    res = np.empty((26, 21), dtype='int')
    for i in range(26):
        for j in range(21):
            res[i, j] = mandel_numba(complex(r1[i], r2[j]))
    return res

In [260]:
%timeit mandelperf_numba()

The slowest run took 2568.37 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 156 µs per loop


In [268]:
%%cython -a

cimport cython
import numpy as np
cimport numpy as np

cdef extern from "complex.h":
    double cabs(double complex)

cdef int mandel_cython(double complex z):
    cdef int n
    cdef int max_iter
    cdef double complex c
    maxiter = 80
    c = z
    for n in range(maxiter):
        if cabs(z) > 2:
            return n
        z = z*z + c
    return maxiter

@cython.boundscheck(False)
@cython.wraparound(False)
def mandelperf_cython():
    cdef int i, j

    cdef double[:] r1 = np.linspace(-2.0, 0.5, 26)
    cdef double[:] r2 = np.linspace(-1.0, 1.0, 21)
    cdef int[:,:] res = np.empty((26,21), dtype=np.int32)

    for i in range(26):
        for j in range(21):
            res[i,j] = mandel_cython(r1[i] + r2[j]*1j)
    return res

In [269]:
%timeit mandelperf_cython()

1000 loops, best of 3: 222 µs per loop


In [271]:
report([mandelperf, mandelperf_numba, mandelperf_cython])

mandelperf: 1.0
mandelperf_numba: 19.7
mandelperf_cython: 15.4


In [454]:
for f in mandelperf, mandelperf_numba, mandelperf_cython:
    print(np.reshape(f(), (26,21)), "\n")

[[ 0  0  0  0  0  0  0  0  0  0 80  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  1  2  2  3  3 80  3  3  2  2  1  1  0  0  0  0]
 [ 0  0  1  1  2  2  2  3  3  3 80  3  3  3  2  2  2  1  1  0  0]
 [ 1  1  2  2  2  2  2  3  3  5 80  5  3  3  2  2  2  2  2  1  1]
 [ 1  2  2  2  2  2  2  3  4  5 80  5  4  3  2  2  2  2  2  2  1]
 [ 1  2  2  2  2  2  3  4  4  6 80  6  4  4  3  2  2  2  2  2  1]
 [ 2  2  2  2  2  2  4  4  5 11 80 11  5  4  4  2  2  2  2  2  2]
 [ 2  2  2  2  2  3  6  6  7 13 80 13  7  6  6  3  2  2  2  2  2]
 [ 2  2  2  2  2  4  6 15 17 80 80 80 17 15  6  4  2  2  2  2  2]
 [ 2  2  2  2  3  4  6 11 80 80 80 80 80 11  6  4  3  2  2  2  2]
 [ 2  2  2  3  3  4  6 34 80 80 80 80 80 34  6  4  3  3  2  2  2]
 [ 2  2  3  3  4  4  6 10 80 80 80 80 80 10  6  4  4  3  3  2  2]
 [ 2  3  3  3  4  5  6  8 14 80 80 80 14  8  6  5  4  3  3  3  2]
 [ 2  3  3  4  5  7 10 23 80 80 80 80 80 23 10  7  5  4  3  3  2]
 [ 3  3  3  6 11 11 80 80 80 80 80 80 80 80 80 11 11  6  3  3  3]
 [ 3  3  4

Pi_sum
----

In [272]:
def pisum(t):
    sum = 0.0
    for j in range(1, 501):
        sum = 0.0
        for k in range(1, t+1):
            sum += 1.0/(k*k)
    return sum

In [273]:
%timeit pisum(10000)

1 loops, best of 3: 1.33 s per loop


In [282]:
@jit
def pisum_numba(t):
    sum = 0.0
    for j in range(1, 501):
        sum = 0.0
        for k in range(1, t+1):
            sum += 1.0/(k*k)
    return sum

In [283]:
%timeit pisum_numba(10000)

10 loops, best of 3: 35.8 ms per loop


In [298]:
%%cython -a

cimport cython

@cython.cdivision(True)
def pisum_cython(int t):
    cdef double sum
    cdef int j, k

    for j in range(1, 501):
        sum = 0.0
        for k in range(1, t+1):
            sum += 1.0/(k*k)
    return sum

In [299]:
%timeit pisum_cython(10000)

10 loops, best of 3: 35.8 ms per loop


In [286]:
report([pisum, pisum_numba, pisum_cython], 10000)

pisum: 1.0
pisum_numba: 35.6
pisum_cython: 35.6


In [455]:
for f in pisum, pisum_numba, pisum_cython:
    print(f(10000))

1.6448340718480652
1.6448340718480652
1.6448340718480652


Rand_mat_stat
----

In [287]:
def randmatstat(t):
    n = 5
    v = np.zeros(t)
    w = np.zeros(t)
    for i in range(1,t):
        a = np.random.randn(n, n)
        b = np.random.randn(n, n)
        c = np.random.randn(n, n)
        d = np.random.randn(n, n)
        P = np.matrix(np.hstack((a, b, c, d)))
        Q = np.matrix(np.vstack((np.hstack((a, b)), np.hstack((c, d)))))
        v[i] = np.trace(np.linalg.matrix_power(np.transpose(P)*P, 4))
        w[i] = np.trace(np.linalg.matrix_power(np.transpose(Q)*Q, 4))
    return (np.std(v)/np.mean(v), np.std(w)/np.mean(w))

In [288]:
%timeit randmatstat(1000)

1 loops, best of 3: 260 ms per loop


In [391]:
def randmatstat_numba(t):
    n = 5
    v = np.zeros(t)
    w = np.zeros(t)
    for i in range(1,t):
        a, b, c, d = np.random.randn(4, n, n)
        P = np.hstack((a, b, c, d))
        Q = np.vstack((np.hstack((a, b)), np.hstack((c, d))))

        PTP = P.T.dot(P)
        QTQ = Q.T.dot(Q)

        v[i] = np.trace(PTP @ PTP @ PTP @ PTP)
        w[i] = np.trace(QTQ @ QTQ @ QTQ @ QTQ)
    return (np.std(v)/np.mean(v), np.std(w)/np.mean(w))

In [393]:
def randmatstat_alt(t):
    n = 5
    v = np.zeros(t)
    w = np.zeros(t)

    for i in range(1,t):
        a, b, c, d = np.random.randn(4, n, n)
        P = np.hstack((a, b, c, d))
        Q = np.vstack((np.hstack((a, b)), np.hstack((c, d))))

        P1 = P.T @ P
        P2 = P1 @ P1
        P4 = P2 @ P2

        Q1 = Q.T @ Q
        Q2 = Q1 @ Q1
        Q4 = Q2 @ Q2
        
        v[i] = np.trace(P4)
        w[i] = np.trace(Q4)

    return (np.std(v)/np.mean(v), np.std(w)/np.mean(w))

In [394]:
%timeit randmatstat_alt(1000)

10 loops, best of 3: 131 ms per loop


In [396]:
report([randmatstat, randmatstat_alt], 1000)

randmatstat: 1.0
randmatstat_alt: 2.1


In [458]:
for f in randmatstat, randmatstat_alt:
    np.random.seed(123)
    print(f(1000))

(0.72398346943567571, 0.7713214961017032)
(0.72398346943567571, 0.7713214961017032)


Rand_mat_mul
----

In [289]:
def randmatmul(n):
    A = np.random.rand(n,n)
    B = np.random.rand(n,n)
    return np.dot(A,B)

In [290]:
%timeit randmatmul(1000)

10 loops, best of 3: 81 ms per loop


Comparison
----

These benchmarks don't mean very much - they were constructed by the Julia team presumably to show off Julia's strengths, and are used here just to illustrate basic Python optimization techniques. In some case (quicksort, mandel, pi_sum), this gives dramatic performance improvements but in other cases (parse_int, rand_mat_stat) there is less improvement. For fib, the speed depends on the algorithm used for calculation, and whether caching is enabled in the recursive case. The final test rand_mat_mul basically depends on the linear algebra library installed (blas, mkl, atlas) and is not really a comparison across languages.

Julia certainly does look promising, and we might include mix in some Julia code in future courses.

In [459]:
benchmarks['P2J'] = benchmarks.Python / benchmarks.Julia
benchmarks['SU'] = [1919.0, 3.2, 25.6, 19.7, 35.6, 2.1, 1.0]
pd.options.display.float_format = '{:.2f}'.format
benchmarks.ix[:, [0,2,3,-2,-1]]

,,Julia,Python,P2J,SU
1,fib,2.11,77.76,36.85,1919.00
2,parse_int,1.45,17.02,11.74,3.20
3,quicksort,1.15,32.89,28.60,25.60
4,mandel,0.79,15.32,19.39,19.70
5,pi_sum,1.00,21.99,21.99,35.60
6,rand_mat_stat,1.66,17.93,10.80,2.10
7,rand_mat_mul,1.02,1.14,1.12,1.00
